In [1]:
import torch
import torchvision
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from dset import AgeDataset

In [3]:
a = torchvision.models.resnet50(pretrained=True)
a.

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/duboy/.torch/models/resnet50-19c8e357.pth
100.0%


In [2]:
#parameters
param = {}
param["l_rate"] = 0.001
param["epoch"] = 10
param["l2_rate"] = 0.00001
param["layers"] = [100,50,20]
param["drop"] = False
param["drop_p"] = 0.1
param["base_drop"] = True
param["base_drop_p"] = 0.2
param["batch_size"] = 10


In [3]:
#auxilary functions
def calc_acc(pred,truth):
    pred =pred.data.view(-1).numpy()
    truth=truth.view(-1).numpy()
    assert pred.shape==truth.shape, "Shapes are not same"
    return (np.abs(pred-truth) < 10).sum() / pred.shape[0]

In [4]:
x_train = np.load("./original/train.npy")
y_train = np.load("./original/train_gt.npy")
x_val = np.load("./original/valid.npy")
y_val = np.load("./original/valid_gt.npy")
print(y_train.shape)

(5000,)


In [5]:
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train).view(-1,1).float()
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val).view(-1,1).float()

In [6]:
#age_dataset = TensorDataset(x_train,y_train)
#age_dataset_val = TensorDataset(x_val,y_val)
age_dataset = AgeDataset()
dataloader = DataLoader(age_dataset, batch_size=param["batch_size"],
                        shuffle=True, num_workers=4)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.res18 = torchvision.models.resnet18(pretrained=True)
        self.res18.eval()
        self.avgpool_layer =self.res18._modules.get('avgpool')
        
        l = param["layers"]
        if(len(l)==0):
            self.fc1 = nn.Linear(512,1)
        elif(len(l)==1):
            self.fc1 = nn.Linear(512,l[0])
            self.fc2 = nn.Linear(l[0],1)
        elif(len(l)==2):
            self.fc1 = nn.Linear(512,l[0])
            self.fc2 = nn.Linear(l[0],l[1])
            self.fc3 = nn.Linear(l[1],1)
        elif(len(l)==3):
            self.fc1 = nn.Linear(512,l[0])
            self.fc2 = nn.Linear(l[0],l[1])
            self.fc3 = nn.Linear(l[1],l[2])
            self.fc4 = nn.Linear(l[2],1)
        if param["drop"]==True :
            self.drop = nn.Dropout(p=param["drop_p"])
            
        if param["base_drop"]==True :
            self.base_drop = nn.Dropout(p=param["base_drop_p"])
        
    def forward(self,x):
        ##first extract feature
        my_embedding = torch.zeros(param["batch_size"], 512, 1, 1)
        def fun(m, i, o): 
            my_embedding.copy_(o.data)
        h = self.avgpool_layer.register_forward_hook(fun)
        h_x = self.res18(x)
        h.remove()
        x = my_embedding.view(param["batch_size"], 512)
        l = param["layers"]
        if param["base_drop"]==True :
            x = self.base_drop(x)
        if(len(l)==0):
            x = self.fc1(x)
        elif(len(l)==1):
            x = F.relu(self.fc1(x))
            if param["drop"]==True :
                x=self.drop(x)
            x = self.fc2(x)
        elif(len(l)==2):
            x = F.relu(self.fc1(x))
            if param["drop"]==True :
                x=self.drop(x)
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
        elif(len(l)==3):
            x = F.relu(self.fc1(x))
            
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)
        return x



Now Training Code

In [8]:
def train(net):
    net.train()
    optimizer = optim.RMSprop(net.parameters(),lr=param["l_rate"], weight_decay=param["l2_rate"])
    criterion = nn.MSELoss()
    log_interval = 5000
    
    for epoch in range(param["epoch"]):
        for batch_idx, (data,target) in enumerate(dataloader):
            data,target = Variable(data),Variable(target)
            #resize
            #data = data.view(-1,512)
            optimizer.zero_grad()
            net_out = net(data)
            loss = criterion(net_out,target)
            
            loss.backward()
            optimizer.step()
        print(batch_idx)
        print('Train Epoch: {}\tLoss: {:.6f}'.format(
                    epoch, loss.data[0]))
        if(epoch>10 and int(loss[0])<60):
            pass
    return int(loss[0])
                
        

In [9]:
#parameters
param = {}
param["l_rate"] = 0.001
param["epoch"] = 10
param["l2_rate"] = 0.00001
param["layers"] = [100,50,20]
param["drop"] = False
param["drop_p"] = 0.1
param["base_drop"] = True
param["base_drop_p"] = 0.2
param["batch_size"] = 10


In [13]:
net = Net()

In [14]:
param["final_loss"] = train(net)
net.eval()
net_out = net(Variable(x_val))
calc_acc(net_out,y_val)

Process Process-6:
Process Process-8:
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-7:
Process Process-5:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f029d83acf8>>
Traceback 

KeyboardInterrupt: 

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 96, in

Now Validation Code